# 3. Use sedona to solve real world problems

Suppose we need to solve the below problems:
- Get the nearest communes of a location
- Get a list of hospitals and doctors(with their geometry location) in Île-de-France
- Get nearest hospitals of a giving point.
- Count hospitals in each commune in Île-de-France.
- Get real route distance and duration between each commune in Île-de-France and giving point




In [1]:
from sedona.spark import *
from pathlib import Path
from pyspark.sql.functions import col, lower, lit, split
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
# get the project root dir
project_root_dir = Path.cwd().parent.parent.parent
print(project_root_dir)

/home/pliu/git/ConstanceDataPlatform


In [3]:
# set the data path
data_dir = f"{project_root_dir}/data"

linux = True
win_root_dir = "C:/Users/PLIU/Documents/ubuntu_share/data_set"
lin_root_dir = "/mnt/hgfs/ubuntu_share/data_set"
if linux:
    data_root_dir = lin_root_dir
else:
    data_root_dir = win_root_dir

fr_commune_file_path = f"{data_root_dir}/kaggle/geospatial/communes_fr_geoparquet"

In [4]:
# build a sedona session (sedona = 1.6.1)
jar_folder = Path(f"{project_root_dir}/jars/sedona-35-213-161")
jar_list = [str(jar) for jar in jar_folder.iterdir() if jar.is_file()]
jar_path = ",".join(jar_list)

# build a sedona session (sedona = 1.6.1) offline
config = SedonaContext.builder() \
    .master("local[*]") \
    .config('spark.jars', jar_path). \
    getOrCreate()

25/01/09 11:00:23 WARN Utils: Your hostname, pliu-ubuntu24 resolves to a loopback address: 127.0.1.1; using 192.168.30.128 instead (on interface ens33)
25/01/09 11:00:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/01/09 11:00:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
# create a sedona context
sedona = SedonaContext.create(config)
sc = sedona.sparkContext
spark = sedona.getActiveSession()


In [6]:
# this sets the encoding of shape files
sc.setSystemProperty("sedona.global.charset", "utf8")

## Q1. Get the nearest communes of a location

To solve this question, we need to:
 
- With a given gps coordinates(WGS84/EPSG:4326), get the bird-view distances between the given point and all commune in France
- Sort the distance in ascending order and show the communes with required numbers


In [7]:
fr_commune_raw = sedona.read.format("geoparquet").load(fr_commune_file_path)

In [8]:
fr_commune_raw.show(5)
fr_commune_raw.printSchema()

+--------------------+--------------------+--------------------+------------+-----+
|            geometry|           wikipedia|             surf_ha|         nom|insee|
+--------------------+--------------------+--------------------+------------+-----+
|POLYGON ((9.32016...|fr:Pie-d'Orezza  ...|     573.00000000...|Pie-d'Orezza|2B222|
|POLYGON ((9.20010...|fr:Lano          ...|     824.00000000...|        Lano|2B137|
|POLYGON ((9.27757...|fr:Cambia        ...|     833.00000000...|      Cambia|2B051|
|POLYGON ((9.25119...|fr:Érone         ...|     393.00000000...|       Érone|2B106|
|POLYGON ((9.28339...|fr:Oletta        ...|    2674.00000000...|      Oletta|2B185|
+--------------------+--------------------+--------------------+------------+-----+
only showing top 5 rows

root
 |-- geometry: geometry (nullable = true)
 |-- wikipedia: string (nullable = true)
 |-- surf_ha: string (nullable = true)
 |-- nom: string (nullable = true)
 |-- insee: string (nullable = true)



In [9]:
fr_commune_df = fr_commune_raw.select("geometry","nom","insee").withColumn("name",lower(col("nom"))).drop("nom")

In [10]:
fr_commune_df.show(5)
fr_commune_df.printSchema()

+--------------------+-----+------------+
|            geometry|insee|        name|
+--------------------+-----+------------+
|POLYGON ((9.32016...|2B222|pie-d'orezza|
|POLYGON ((9.20010...|2B137|        lano|
|POLYGON ((9.27757...|2B051|      cambia|
|POLYGON ((9.25119...|2B106|       érone|
|POLYGON ((9.28339...|2B185|      oletta|
+--------------------+-----+------------+
only showing top 5 rows

root
 |-- geometry: geometry (nullable = true)
 |-- insee: string (nullable = true)
 |-- name: string (nullable = true)



In [11]:
temp_table_name="fr_commune"
fr_commune_df.createOrReplaceTempView(temp_table_name)

In [12]:
def get_nearest_commune(latitude:str, longitude:str, max_commune_number:int):
    """
    This function calculates the nearest commune to the given latitude and longitude.
    :param latitude: latitude of the given point
    :param longitude: longitude of the given point
    :param max_commune_number: Specify the max number of commune in the result
    :return: 
    """
    nearest_commune_df = sedona.sql(f"""
     SELECT z.name as commune_name, z.insee, ST_DistanceSphere(ST_PointFromText('{longitude},{latitude}', ','), z.geometry) AS distance FROM {temp_table_name} as z ORDER BY distance ASC LIMIT {max_commune_number}
     """)
    return nearest_commune_df

In [13]:
# the gps coordinates for casd is 48.8190155° N, 2.3081911° E

casd_latitude = "48.8190155"
casd_longitude = "2.3081911"

casd_geo = f"POINT({casd_longitude} {casd_latitude})"

In [14]:
kb_nearest_shape_df = get_nearest_commune(casd_latitude,casd_longitude,36000)

In [15]:
%%time

kb_nearest_shape_df.show(50)
kb_nearest_shape_df.count()

+--------------------+-----+------------------+
|        commune_name|insee|          distance|
+--------------------+-----+------------------+
|           montrouge|92049| 782.4225524838963|
|            malakoff|92046| 931.0760610707576|
|              vanves|92075|1548.4653721969821|
|           châtillon|92020|2244.2297097020064|
|             bagneux|92007|2306.6895398469915|
|             arcueil|94003|2420.7221725908407|
|            gentilly|94037| 2712.776684694567|
| issy-les-moulineaux|92040|3212.5435965134634|
|              cachan|94016| 3506.334512834338|
|  fontenay-aux-roses|92032| 3619.863974489254|
|  le kremlin-bicêtre|94043| 3670.315588897698|
|      bourg-la-reine|92014| 4337.380330327545|
|             clamart|92023| 4677.596275760091|
|           villejuif|94076| 4806.259641317598|
|              sceaux|92071| 4807.635098158753|
|               paris|75056| 4891.894805477514|
|     l'haÿ-les-roses|94038| 5227.715047978959|
|boulogne-billancourt|92012| 5418.541958

CPU times: user 11.5 ms, sys: 5.02 ms, total: 16.5 ms
Wall time: 7.25 s


34955

In [16]:
# the gps coordinates for Paul-Brousse is 48.7951606539 N, 2.3636935981 E
pb_latitude = "48.7951606539"
pb_longitude = "2.3636935981"
pb_geo = f"POINT({pb_longitude} {pb_latitude})"

In [17]:
pb_nearest_shape_df = get_nearest_commune(pb_latitude,pb_longitude,36000)

In [18]:
%%time

pb_nearest_shape_df.show(10)
pb_nearest_shape_df.count()

+------------------+-----+------------------+
|      commune_name|insee|          distance|
+------------------+-----+------------------+
|         villejuif|94076|417.13873642972163|
|le kremlin-bicêtre|94043|1616.4950323944458|
|            cachan|94016|2350.1367372643913|
|   vitry-sur-seine|94081|2391.3950596721934|
|           arcueil|94003|2455.2075433323284|
|          gentilly|94037|2469.3891101286713|
|    ivry-sur-seine|94041|2592.6844198231634|
|   l'haÿ-les-roses|94038|2842.8348963659664|
|    chevilly-larue|94021|3239.0306951319963|
|    bourg-la-reine|92014|3788.6932471326836|
+------------------+-----+------------------+
only showing top 10 rows



CPU times: user 8.43 ms, sys: 3.91 ms, total: 12.3 ms
Wall time: 4.76 s


34955

## Q2. Get a list of hospitals and doctors(with their geometry location) in Île-de-France

To get the list of hospitals and doctors in Île-de-France, we use the (OSM)Open Street Map sample data.

The sample data which I will use in this notebook can be downloaded from this page: 
https://download.geofabrik.de/europe/france.html. I use the `Ile-de-France` map (`ile-de-france-latest.osm.pbf`)

In [19]:
ile_france_pbf_path=f"{data_root_dir}/geo_spatial/ile-de-france-geo-parquet"
osm_ile_france_df = spark.read.parquet(ile_france_pbf_path)

In [20]:
osm_ile_france_df.show(15)

+------+----+------------------+------------------+-----+---------+--------------------+--------------------+
|    id|type|          latitude|         longitude|nodes|relations|                tags|                info|
+------+----+------------------+------------------+-----+---------+--------------------+--------------------+
|122626|   0|49.115966300000004|         2.5549119|   []|       []|                  {}|{3, 2020-05-10 11...|
|122627|   0|49.110294100000004|         2.5521725|   []|       []|                  {}|{4, 2009-02-13 19...|
|122631|   0|        49.0834393|2.5511375000000003|   []|       []|                  {}|{15, 2021-06-30 1...|
|122632|   0|        49.0675225|2.5524679000000003|   []|       []|                  {}|{17, 2019-04-10 1...|
|122633|   0|         49.063616|2.5522412000000005|   []|       []|                  {}|{17, 2009-02-13 1...|
|122634|   0|        49.0597465|2.5509097000000005|   []|       []|                  {}|{2, 2009-02-13 19...|
|122635|  

In [21]:
osm_ile_france_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- type: byte (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- nodes: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- relations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- relationType: byte (nullable = true)
 |    |    |-- role: string (nullable = true)
 |-- tags: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- info: struct (nullable = true)
 |    |-- version: integer (nullable = true)
 |    |-- timestamp: timestamp (nullable = true)
 |    |-- changeset: long (nullable = true)
 |    |-- userId: integer (nullable = true)
 |    |-- userName: string (nullable = true)
 |    |-- visible: boolean (nullable = true)



In [22]:
def show_row_details(entity_id:int):
    sample_row = osm_ile_france_df.filter(osm_ile_france_df.id == entity_id )
    sample_row.show(truncate=False, vertical=True)

In [23]:
hospital_df = osm_ile_france_df.select("id", "latitude", "longitude", "tags").where("element_at(tags, 'amenity') in ('hospital', 'clinic')")

In [24]:
doctor_df = osm_ile_france_df.select("id", "latitude", "longitude", "tags").where("element_at(tags, 'amenity') == 'doctors'")

In [25]:
print(f"total hosptitals number in ile-de-france: {hospital_df.count()}")

total hosptitals number in ile-de-france: 564


In [26]:
hospital_df.show(5)

+---------+------------------+------------------+--------------------+
|       id|          latitude|         longitude|                tags|
+---------+------------------+------------------+--------------------+
|452401907| 48.78635369999983| 2.291711499999987|{amenity -> hospi...|
|476313165|48.878517699999826| 2.414601899999999|{name -> Maternit...|
|483569726| 48.82353579999984|2.2768316999999985|{website -> https...|
|670633220|48.722277100000056|2.4525995000000083|{name -> Centre H...|
|783760856| 48.83526160000006|         2.2442898|{name -> Centre d...|
+---------+------------------+------------------+--------------------+
only showing top 5 rows



In [27]:
show_row_details(670633220)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 id        | 670633220                                                                                                                                                                                                                                                                                          

In [27]:
print(f"total doctors number in ile-de-france: {doctor_df.count()} ")

total doctors number in ile-de-france: 1298 


In [28]:
doctor_df.show(5)

+---------+------------------+------------------+--------------------+
|       id|          latitude|         longitude|                tags|
+---------+------------------+------------------+--------------------+
|293986528| 48.86378410000007|2.3814177000000036|{amenity -> docto...|
|302305751|        48.5833366|2.2414200000000104|{amenity -> docto...|
|416708946|48.872033599999774|2.3765707000000202|{website -> https...|
|456140610| 48.75835359999986| 3.048538999999998|{amenity -> docto...|
|477197148|49.010437500000016|2.0296950999999948|{amenity -> doctors}|
+---------+------------------+------------------+--------------------+
only showing top 5 rows



In [29]:
show_row_details(416708946)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 id        | 416708946                                                                                                                                                                                                                                                
 type      | 0                                                                                                                                                                                                                                                        
 latitude  | 48.872033599999774                                                                                                                                                                                    

In [30]:
from pyspark.sql import DataFrame


def geo_df_convertor(source_df: DataFrame, lat_col_name: str, long_col_name: str, source_epsg_code: str,
                     target_epsg_code: str, target_geo_col_name: str = "location"):
    """
    This function takes a dataframe with gps coordinate column(string type), convert the string column to a geometry column. The returned dataframe can be stored as geoparquet.
    :param source_df: 
    :type source_df: 
    :param long_col_name: longitude column name
    :type long_col_name: str
    :param lat_col_name: latitude column name
    :type lat_col_name: str
    :param source_epsg_code: the csr code of the input gps coordinates
    :type source_epsg_code: str
    :param target_epsg_code: the csr code of the output gps coordinates
    :type target_epsg_code: str
    :param target_geo_col_name: 
    :type target_geo_col_name: 
    :return: 
    :rtype: 
    """
    # create a temp view of the source df, the table name is the name of the data frame
    temp_table_name = "raw_source_df"
    source_df.createOrReplaceTempView(f"{temp_table_name}")
    target_df = sedona.sql(f"""
    SELECT id, tags,
    ST_Transform(ST_Point(CAST({long_col_name} AS Decimal(24,20)), CAST({lat_col_name} AS Decimal(24,20))), '{source_epsg_code}', '{target_epsg_code}') AS {target_geo_col_name} from {temp_table_name}""")
    return target_df


def geo_table_convertor(source_table_name: str, lat_col_name: str, long_col_name: str, source_epsg_code: str,
                        target_epsg_code: str, target_geo_col_name: str = "location"):
    """
    This function takes a spark temp view with gps coordinate column(string type), convert the string column to a geometry column. The returned dataframe can be stored as geoparquet.
    :param source_table_name: 
    :type source_table_name: 
    :param target_geo_col_name: 
    :type target_geo_col_name: 
    :param long_col_name: 
    :type long_col_name: 
    :param lat_col_name: 
    :type lat_col_name: 
    :param source_epsg_code: 
    :type source_epsg_code: 
    :param target_epsg_code: 
    :type target_epsg_code: 
    :return: 
    :rtype: 
    """
    # create a temp view of the source df, the table name is the name of the data frame
    target_df = sedona.sql(f"""
    SELECT 
    ST_Transform(ST_Point(CAST({long_col_name} AS Decimal(24,20)), CAST({lat_col_name} AS Decimal(24,20))), '{source_epsg_code}', '{target_epsg_code}') AS {target_geo_col_name} from {source_table_name}""")
    return target_df


## Q3. Get nearest hospitals of a giving point.

The osm sample data does not provide geometry column directly, so the first step is to build a geometry column with the giving gps coordinates. 

In [31]:
# Set up the epsg code value, osm uses epsg:25832
source_epsg_code = "epsg:4326"
# eu centered epsg code, more information can be found https://epsg.io/25832
target_epsg_code = "epsg:4326"

hospital_geo_table_name = "hospital_geo"
hospital_geo_df = geo_df_convertor(hospital_df,"latitude","longitude",source_epsg_code,target_epsg_code)
hospital_geo_df.cache()
hospital_geo_df.createOrReplaceTempView(hospital_geo_table_name)

In [32]:
hospital_geo_df.show(5)

+---------+--------------------+--------------------+
|       id|                tags|            location|
+---------+--------------------+--------------------+
|452401907|{amenity -> hospi...|POINT (2.29171149...|
|476313165|{name -> Maternit...|POINT (2.41460189...|
|483569726|{website -> https...|POINT (2.27683169...|
|670633220|{name -> Centre H...|POINT (2.45259950...|
|783760856|{name -> Centre d...|POINT (2.2442898 ...|
+---------+--------------------+--------------------+
only showing top 5 rows



In [34]:
hospital_geo_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- tags: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- location: geometry (nullable = true)



In [34]:
def get_near_hospital(patient_loc:str, distance:float):
    """
    This function get the nearest hospital based on distance with a given patient location
    :param patient_loc: gps coordinates in format "POINT(longitude, latitude)"
    :param distance: the max distance between hospital and patient
    :return: 
    """
    tmp_df = sedona.sql(f"""
                         SELECT 
                         hospital_geo.id, 
                         hospital_geo.tags,
                         ST_AsGeoJSON(ST_Transform(hospital_geo.location, '{target_epsg_code}', 'epsg:4326')) hospital_point, 
                         ST_DistanceSphere(ST_GeomFromWKT('{patient_loc}'), hospital_geo.location) distance_meter
                         FROM hospital_geo
                         order by distance_meter asc;
""")
    near_hospital_df= tmp_df.filter(tmp_df.distance_meter<=distance)
    return near_hospital_df

To better understand the query, we use this [website](https://www.keene.edu/campus/maps/tool/) 
to show gps coordinates on a map.

CASD coordinates:
```text
2.3081911,48.8190155
```

Paul-Brousse coordinates:
```text
2.3636935981,48.7951606539
```



In [35]:
print(f"CASD coordinates: {casd_geo}")
print(f"Paul-Brousse coordinates: {pb_geo}")

CASD coordinates: POINT(2.3081911 48.8190155)
Paul-Brousse coordinates: POINT(2.3636935981 48.7951606539)


### Below codes returns the nearest hospital of CASD

In [38]:
# get all hospital within 5000 meter of CASD
casd_near_hos_df= get_near_hospital(casd_geo, 15000)
casd_near_hos_df.show(5)

+-----------+--------------------+--------------------+------------------+
|         id|                tags|      hospital_point|    distance_meter|
+-----------+--------------------+--------------------+------------------+
| 2506232459|{website -> http:...|{"type":"Point","...| 999.4514393892221|
| 4936398378|{amenity -> clini...|{"type":"Point","...|1043.5714649372987|
| 9073838043|{healthcare:speci...|{"type":"Point","...|1365.4667383528551|
|10883369653|{name -> Clinique...|{"type":"Point","...|1746.0477473786032|
|  483569726|{website -> https...|{"type":"Point","...|2350.2594029700895|
+-----------+--------------------+--------------------+------------------+
only showing top 5 rows



In [39]:
casd_near_hos_df.count()

154

In [38]:
print(f"Hospital count within 5km radius of CASD: {casd_near_hos_df.count()} ")

Hospital count within 5km radius of CASD: 38 


In [39]:
show_row_details(136)

(0 rows)



In [40]:
casd_near_hos_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- tags: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- hospital_point: string (nullable = true)
 |-- distance_meter: double (nullable = true)



In [41]:
kepler_map_path = f"{data_dir}/tmp/casd_near_hospital_map.html"

sedona_kepler_map = SedonaKepler.create_map(df=casd_near_hos_df,name="casd_near_hospital")

sedona_kepler_map.save_to_html(file_name=kepler_map_path)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to /home/pliu/git/ConstanceDataPlatform/data/tmp/casd_near_hospital_map.html!


In [42]:
# get all hospital within 5000 meter of Paul-Brousse
pb_near_hos_df= get_near_hospital(pb_geo, 5000)
pb_near_hos_df.show(5)

+-----------+--------------------+--------------------+------------------+
|         id|                tags|      hospital_point|    distance_meter|
+-----------+--------------------+--------------------+------------------+
|10926295628|{amenity -> clini...|{"type":"Point","...|1054.1612866076684|
| 8860878651|{amenity -> hospi...|{"type":"Point","...| 1493.186653292936|
| 9247842729|{amenity -> clini...|{"type":"Point","...|2409.7512937880547|
| 9371510917|{website -> https...|{"type":"Point","...|2503.1995985912217|
| 9371501687|{name -> Service ...|{"type":"Point","...| 2885.792668156662|
+-----------+--------------------+--------------------+------------------+
only showing top 5 rows



In [43]:
print(f"Hospital count within 5km radius of Paul-Brousse: {pb_near_hos_df.count()} ")

Hospital count within 5km radius of Paul-Brousse: 24 


In [44]:
show_row_details(10926295628)

-RECORD 0-----------------------------------------------------------------------------
 id        | 10926295628                                                              
 type      | 0                                                                        
 latitude  | 48.78587790000029                                                        
 longitude | 2.366615800000005                                                        
 nodes     | []                                                                       
 relations | []                                                                       
 tags      | {amenity -> clinic, healthcare -> clinic, name -> Centre médical Aragon} 
 info      | {1, 2023-05-25 15:06:07, 0, 0, , NULL}                                   



In [45]:
kepler_map_path = f"{data_dir}/tmp/pb_near_hospital_map.html"

sedona_kepler_map = SedonaKepler.create_map(df=casd_near_hos_df,name="pb_near_hospital")

sedona_kepler_map.save_to_html(file_name=kepler_map_path)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to /home/pliu/git/ConstanceDataPlatform/data/tmp/pb_near_hospital_map.html!


## Q4. Count hospitals in each commune in Île-de-France

For the sake of simplicity, here we only choose the districts(communes) in a radius of 8000 meters of the paris center.

1. Get the centroid of Paris
2. Get all commune in a Radius of 8000 meters from the centroid of Paris
3. Spatial Join(ST_Contains) the hospital data frame with the commune data frame
4. Group by the commune and count the hospital numbers
5. Generate a map to visualize the results

In [46]:
from sedona.spark import *
paris_polygon_df= fr_commune_df.filter(col("name")=="paris").withColumn("centroid",ST_Centroid(col("geometry")))
paris_polygon_df.show()
paris_polygon_df.createOrReplaceTempView("paris_polygon")


+--------------------+-----+-----+--------------------+
|            geometry|insee| name|            centroid|
+--------------------+-----+-----+--------------------+
|POLYGON ((2.22412...|75056|paris|POINT (2.34287643...|
+--------------------+-----+-----+--------------------+



In [47]:
# 2.3428764301940275,48.85662219553845
paris_polygon_df.select("centroid").show(truncate=False)

+--------------------------------------------+
|centroid                                    |
+--------------------------------------------+
|POINT (2.3428764301940275 48.85662219553845)|
+--------------------------------------------+



In [48]:
distance = 8000
query = f"SELECT fr.name FROM paris_polygon p, fr_commune fr WHERE ST_DWithin(p.centroid, fr.geometry,{distance},true)"
filtered_commune_df = sedona.sql(query)

In [49]:
# We get 30 communes in the raius of 8000 meters
print(f"total commune count: {filtered_commune_df.count()}")
filtered_commune_df.show()

25/01/07 14:50:56 WARN DistanceJoinExec: [SedonaSQL] Join dominant side partition number 4 is larger than 1/2 of the dominant side count 1
25/01/07 14:50:56 WARN DistanceJoinExec: [SedonaSQL] Try to use follower side partition number 4
25/01/07 14:50:56 WARN DistanceJoinExec: [SedonaSQL] Join follower side partition number is also larger than 1/2 of the dominant side count 1
25/01/07 14:50:56 WARN DistanceJoinExec: [SedonaSQL] Try to use 1/2 of the dominant side count 0 as the partition number of both sides
25/01/07 14:50:56 WARN DistanceJoinExec: [SedonaSQL] 1/2 of 0 is equal to 0. Use 1 as the partition number of both sides instead.
25/01/07 14:50:57 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


total commune count: 30


25/01/07 14:51:02 WARN DistanceJoinExec: [SedonaSQL] Join dominant side partition number 4 is larger than 1/2 of the dominant side count 1
25/01/07 14:51:02 WARN DistanceJoinExec: [SedonaSQL] Try to use follower side partition number 4
25/01/07 14:51:02 WARN DistanceJoinExec: [SedonaSQL] Join follower side partition number is also larger than 1/2 of the dominant side count 1
25/01/07 14:51:02 WARN DistanceJoinExec: [SedonaSQL] Try to use 1/2 of the dominant side count 0 as the partition number of both sides
25/01/07 14:51:02 WARN DistanceJoinExec: [SedonaSQL] 1/2 of 0 is equal to 0. Use 1 as the partition number of both sides instead.
25/01/07 14:51:03 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


+--------------------+
|                name|
+--------------------+
|boulogne-billancourt|
|           châtillon|
|    levallois-perret|
|              vanves|
|               paris|
|           vincennes|
|            bagnolet|
|         romainville|
|   neuilly-sur-seine|
|           les lilas|
|         saint-mandé|
|le pré-saint-gervais|
| issy-les-moulineaux|
|          courbevoie|
|           montreuil|
|       aubervilliers|
|saint-ouen-sur-seine|
|   charenton-le-pont|
|              pantin|
|      ivry-sur-seine|
+--------------------+
only showing top 20 rows



In [50]:
target_commune = [row["name"] for row in filtered_commune_df.select("name").collect()]

print(target_commune)

25/01/07 14:51:08 WARN DistanceJoinExec: [SedonaSQL] Join dominant side partition number 4 is larger than 1/2 of the dominant side count 1
25/01/07 14:51:08 WARN DistanceJoinExec: [SedonaSQL] Try to use follower side partition number 4
25/01/07 14:51:08 WARN DistanceJoinExec: [SedonaSQL] Join follower side partition number is also larger than 1/2 of the dominant side count 1
25/01/07 14:51:08 WARN DistanceJoinExec: [SedonaSQL] Try to use 1/2 of the dominant side count 0 as the partition number of both sides
25/01/07 14:51:08 WARN DistanceJoinExec: [SedonaSQL] 1/2 of 0 is equal to 0. Use 1 as the partition number of both sides instead.
25/01/07 14:51:09 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


['boulogne-billancourt', 'châtillon', 'levallois-perret', 'vanves', 'paris', 'vincennes', 'bagnolet', 'romainville', 'neuilly-sur-seine', 'les lilas', 'saint-mandé', 'le pré-saint-gervais', 'issy-les-moulineaux', 'courbevoie', 'montreuil', 'aubervilliers', 'saint-ouen-sur-seine', 'charenton-le-pont', 'pantin', 'ivry-sur-seine', 'asnières-sur-seine', 'gentilly', 'le kremlin-bicêtre', 'arcueil', 'villejuif', 'cachan', 'malakoff', 'montrouge', 'clichy-la-garenne', 'bagneux']


In [51]:
target_commune_df = fr_commune_df.filter(col("name").isin(target_commune))
target_commune_df.show()
target_commune_df.createOrReplaceTempView("target_commune")

+--------------------+-----+--------------------+
|            geometry|insee|                name|
+--------------------+-----+--------------------+
|POLYGON ((3.26578...|02043|             bagneux|
|POLYGON ((3.08236...|03069|           châtillon|
|POLYGON ((3.79568...|51032|             bagneux|
|POLYGON ((5.69078...|39122|           châtillon|
|POLYGON ((5.84683...|54041|             bagneux|
|POLYGON ((4.62395...|69050|           châtillon|
|POLYGON ((3.16833...|03015|             bagneux|
|POLYGON ((2.22279...|92012|boulogne-billancourt|
|POLYGON ((1.75451...|62588|           montreuil|
|POLYGON ((1.71683...|36011|             bagneux|
|POLYGON ((1.35566...|28267|           montreuil|
|POLYGON ((-0.8686...|85148|           montreuil|
|POLYGON ((2.27139...|92020|           châtillon|
|POLYGON ((2.27102...|92044|    levallois-perret|
|POLYGON ((2.27261...|92075|              vanves|
|POLYGON ((2.22412...|75056|               paris|
|POLYGON ((2.41844...|94080|           vincennes|


In [52]:
commune_hospital_df = sedona.sql(f"""
 select 
 target_commune.name, target_commune.geometry, hospital_geo.id
 FROM target_commune, hospital_geo 
 WHERE 
 ST_Contains(target_commune.geometry, hospital_geo.location)
""")

commune_hospital_df.show()
commune_hospital_df.createOrReplaceTempView("commune_hospital")

+--------------------+--------------------+-----------+
|                name|            geometry|         id|
+--------------------+--------------------+-----------+
|boulogne-billancourt|POLYGON ((2.22279...| 3189803775|
|boulogne-billancourt|POLYGON ((2.22279...|  783760856|
|boulogne-billancourt|POLYGON ((2.22279...| 1224158095|
|    levallois-perret|POLYGON ((2.27102...| 5719766166|
|               paris|POLYGON ((2.22412...|10883369653|
|               paris|POLYGON ((2.22412...| 4936398378|
|               paris|POLYGON ((2.22412...| 7923610571|
|               paris|POLYGON ((2.22412...| 9073838043|
|               paris|POLYGON ((2.22412...|10061970658|
|               paris|POLYGON ((2.22412...| 8269359942|
|               paris|POLYGON ((2.22412...| 1362787029|
|               paris|POLYGON ((2.22412...| 9247842736|
|               paris|POLYGON ((2.22412...| 4657177274|
|               paris|POLYGON ((2.22412...| 3624753425|
|               paris|POLYGON ((2.22412...| 7568

In [53]:
hospital_count_df = sedona.sql("SELECT c.name, c.geometry, count(*) as hospital_count FROM commune_hospital c GROUP BY c.name, c.name, c.geometry sort by hospital_count desc")

In [54]:
hospital_count_df.show(5)

+--------------------+--------------------+--------------+
|                name|            geometry|hospital_count|
+--------------------+--------------------+--------------+
|               paris|POLYGON ((2.22412...|            87|
|   neuilly-sur-seine|POLYGON ((2.24562...|             4|
|boulogne-billancourt|POLYGON ((2.22279...|             3|
|          courbevoie|POLYGON ((2.23059...|             3|
|   clichy-la-garenne|POLYGON ((2.28737...|             3|
+--------------------+--------------------+--------------+
only showing top 5 rows



In [55]:
kepler_map_path = f"{data_dir}/tmp/hospital_count_map.html"

sedona_kepler_map = SedonaKepler.create_map(df=hospital_count_df,name="hospital_count_df")

sedona_kepler_map.save_to_html(file_name=kepler_map_path)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


Map saved to /home/pliu/git/ConstanceDataPlatform/data/tmp/hospital_count_map.html!


## 5. Get real route distance and duration

As sedona does not provide this kind of functions, we must define a custom function which calculates the real route distance and duration. Then we register the function in the sedona context so it can be applied on all rows of a dataframe

### 5.1 Define route distance and duration function

This function takes a source and destination coordinates, it returns the route distance and journey duration in car max allow speed. 

In [56]:
import requests
import urllib3
import json
# suppress ssl certificate warning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_route(lat_start:str, long_start:str, lat_end:str, long_end:str, show_steps:str="false")->dict:
    """
    This function takes a starting point and end point gps coordinates, then call the osrm rest api.
    It returns the api json response if the response status is 200, otherwise return None.
    :param lat_start: 
    :type lat_start: 
    :param long_start: 
    :type long_start: 
    :param lat_end: 
    :type lat_end: 
    :param long_end: 
    :type long_end: 
    :param show_steps: 
    :type show_steps: 
    :return: 
    :rtype: 
    """
    host="maps-api.casd.local"
    start_point = f"{long_start},{lat_start}"
    end_point= f"{long_end},{lat_end}"
    # Define the URL
    url = f"https://{host}/route/v1/driving/{start_point};{end_point}?steps={show_steps}"
    
    # Make the GET request
    response = requests.get(url,verify=False)
    json_response = None
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the response content
        json_response = response.json()
    else:
        print("Error:", response.status_code)
    return json_response


def pretty_print(json_str:dict)->None:
    # Pretty-print the JSON
    pretty_json = json.dumps(json_str, indent=4)
    print(pretty_json)
    
    
def parse_route_json(input_route:dict)->(float,float):
    """
    This function parse the orsm json response, and return distance(meter), duration(minute)
    :param input_route: 
    :type input_route: 
    :return: tuple of distance and duration
    :rtype: (float,float)
    """
    route = input_route['routes'][0]
    if route:
        # the raw distance is in meter
        distance = route["distance"]
        # the raw duration is in second
        # the returned duration is in minutes
        duration = round((route["duration"]/60), 2)
    else:
        distance = 0
        duration = 0
    return distance, duration


def calculate_distance_duration(lat_start:str,long_start:str,lat_end:str,long_end:str)->(float,float):
    """
    This function takes a starting point and end point gps coordinates, then call the osrm rest api. It
    parses the response and returns the distance(meter) and duration(minutes)
    :param lat_start: 
    :type lat_start: 
    :param long_start: 
    :type long_start: 
    :param lat_end: 
    :type lat_end: 
    :param long_end: 
    :type long_end: 
    :return: 
    :rtype: 
    """
    route = get_route(lat_start,long_start,lat_end,long_end)
    return parse_route_json(route)

In [57]:
route_between_casd_paul= get_route(casd_latitude,casd_longitude,pb_latitude,pb_longitude,show_steps="true")

In [58]:
pretty_print(route_between_casd_paul)

{
    "code": "Ok",
    "routes": [
        {
            "geometry": "a~}hH_{aMb@gEoR{YcAqDzVinBrIqSdBy_Ao@_GsEiP_A}KnAaKdGgLjA}Wr|A{\\jF_ChNkBfM]^tPiAB",
            "legs": [
                {
                    "steps": [
                        {
                            "geometry": "a~}hH_{aMR_DBMJY",
                            "maneuver": {
                                "bearing_after": 101,
                                "bearing_before": 0,
                                "location": [
                                    2.308476,
                                    48.819052
                                ],
                                "modifier": "straight",
                                "type": "depart"
                            },
                            "mode": "driving",
                            "driving_side": "right",
                            "name": "",
                            "intersections": [
                                {
        

In [59]:
distance, duration = parse_route_json(route_between_casd_paul)
print(f"car distance between CASD:{casd_geo} and Paul-Brousse:{pb_geo} is: {distance} meters.")
print(f"Journey duration between CASD:{casd_geo} and Paul-Brousse:{pb_geo} is: {duration} minutes.")

car distance between CASD:POINT(2.3081911 48.8190155) and Paul-Brousse:POINT(2.3636935981 48.7951606539) is: 6865.1 meters.
Journey duration between CASD:POINT(2.3081911 48.8190155) and Paul-Brousse:POINT(2.3636935981 48.7951606539) is: 10.34 minutes.


## 5.2 Register the function in sedona(spark)

After registration, this function can be applied on each row of a dataframe.

In [60]:
def calculate_distance_duration_str(lat_start:str,long_start:str,lat_end:str,long_end:str) -> str:
    """
    This function is a wrapper of calculate_distance_duration, it returns a string "distance;duration". With one 
    withColumn we can have all information, which will limit the osrm api call
    :param lat_start: 
    :type lat_start: 
    :param long_start: 
    :type long_start: 
    :param lat_end: 
    :type lat_end: 
    :param long_end: 
    :type long_end: 
    :return: 
    :rtype: 
    """
    distance, duration= calculate_distance_duration(lat_start,long_start,lat_end,long_end)
    return f"{distance};{duration}"

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf


@udf(returnType=StringType()) 
def get_distance_duration(lat_start:str,long_start:str,lat_end:str,long_end:str):
    return calculate_distance_duration_str(lat_start,long_start,lat_end,long_end)

In [61]:
print(f"Total commune number before cleaning: {target_commune_df.count()}")

Total commune number before cleaning: 41


25/01/07 14:51:22 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr
25/01/07 14:51:22 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr
25/01/07 14:51:22 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr
25/01/07 14:51:22 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr


In [62]:
target_commune_df.orderBy("name").show(50)

+--------------------+-----+--------------------+
|            geometry|insee|                name|
+--------------------+-----+--------------------+
|POLYGON ((2.31787...|94003|             arcueil|
|POLYGON ((2.26493...|92004|  asnières-sur-seine|
|POLYGON ((2.36560...|93001|       aubervilliers|
|POLYGON ((3.26578...|02043|             bagneux|
|POLYGON ((1.71683...|36011|             bagneux|
|POLYGON ((3.79568...|51032|             bagneux|
|POLYGON ((2.29229...|92007|             bagneux|
|POLYGON ((5.84683...|54041|             bagneux|
|POLYGON ((3.16833...|03015|             bagneux|
|POLYGON ((2.41344...|93006|            bagnolet|
|POLYGON ((2.22279...|92012|boulogne-billancourt|
|POLYGON ((2.31867...|94016|              cachan|
|POLYGON ((2.39025...|94018|   charenton-le-pont|
|POLYGON ((3.08236...|03069|           châtillon|
|POLYGON ((2.27139...|92020|           châtillon|
|POLYGON ((5.69078...|39122|           châtillon|
|POLYGON ((4.62395...|69050|           châtillon|


In [63]:
clean_commune_df = target_commune_df.dropDuplicates(["name"])

In [64]:
print(f"Total commune number after cleaning: {clean_commune_df.count()}")

25/01/07 14:52:01 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr
25/01/07 14:52:01 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr
25/01/07 14:52:01 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr
25/01/07 14:52:01 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr


Total commune number after cleaning: 30


In [65]:
# convert the geometry column to gps coordinates of the commune centroid
clean_commune_df = clean_commune_df.withColumn("centroid",ST_Centroid(col("geometry"))).withColumn("longitude",ST_X(col("centroid"))).withColumn("latitude",ST_Y(col("centroid"))).drop("geometry")
clean_commune_df.show(2)

+-----+------------------+--------------------+------------------+-----------------+
|insee|              name|            centroid|         longitude|         latitude|
+-----+------------------+--------------------+------------------+-----------------+
|94003|           arcueil|POINT (2.33400966...| 2.334009668025032|48.80541926125269|
|92004|asnières-sur-seine|POINT (2.28779307...|2.2877930748988335| 48.9159100346966|
+-----+------------------+--------------------+------------------+-----------------+
only showing top 2 rows



In [66]:
clean_commune_df = clean_commune_df.withColumn("casd_lat", lit(casd_latitude)).withColumn("casd_long",lit(casd_longitude)).drop("centroid")
clean_commune_df.show(2)

+-----+------------------+------------------+-----------------+----------+---------+
|insee|              name|         longitude|         latitude|  casd_lat|casd_long|
+-----+------------------+------------------+-----------------+----------+---------+
|94003|           arcueil| 2.334009668025032|48.80541926125269|48.8190155|2.3081911|
|92004|asnières-sur-seine|2.2877930748988335| 48.9159100346966|48.8190155|2.3081911|
+-----+------------------+------------------+-----------------+----------+---------+
only showing top 2 rows



In [67]:
distance_df = clean_commune_df.withColumn("distance_duration",get_distance_duration(col("casd_lat"),col("casd_long"),col("latitude"),col("longitude")))
distance_df.show(2)

+-----+------------------+------------------+-----------------+----------+---------+-----------------+
|insee|              name|         longitude|         latitude|  casd_lat|casd_long|distance_duration|
+-----+------------------+------------------+-----------------+----------+---------+-----------------+
|94003|           arcueil| 2.334009668025032|48.80541926125269|48.8190155|2.3081911|      3091.1;6.56|
|92004|asnières-sur-seine|2.2877930748988335| 48.9159100346966|48.8190155|2.3081911|    16913.3;21.55|
+-----+------------------+------------------+-----------------+----------+---------+-----------------+
only showing top 2 rows



/home/pliu/git/ConstanceDataPlatform/cdp_venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pliu/git/ConstanceDataPlatform/cdp_venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pliu/git/ConstanceDataPlatform/cdp_venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/

In [68]:
clean_distance_df = (distance_df
                              .withColumn("distance(meter)", split(col("distance_duration"),";")[0].cast("decimal"))
                              .withColumn("duration(minutes)", split(col("distance_duration"),";")[1].cast("decimal"))
                              .drop("distance_duration"))

clean_distance_df.orderBy(col("distance(meter)")).show(5)

/home/pliu/git/ConstanceDataPlatform/cdp_venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pliu/git/ConstanceDataPlatform/cdp_venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pliu/git/ConstanceDataPlatform/cdp_venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/

+-----+---------+------------------+-----------------+----------+---------+---------------+-----------------+
|insee|     name|         longitude|         latitude|  casd_lat|casd_long|distance(meter)|duration(minutes)|
+-----+---------+------------------+-----------------+----------+---------+---------------+-----------------+
|92049|montrouge|2.3171758940549156|48.81520615999795|48.8190155|2.3081911|            990|                2|
|92046| malakoff|2.2960333939469506|48.81656028297312|48.8190155|2.3081911|           1842|                4|
|92075|   vanves|2.2873917410939963|48.82154081411746|48.8190155|2.3081911|           2010|                4|
|94003|  arcueil| 2.334009668025032|48.80541926125269|48.8190155|2.3081911|           3091|                7|
|94037| gentilly|2.3442035303717184|48.81328243443601|48.8190155|2.3081911|           3842|                6|
+-----+---------+------------------+-----------------+----------+---------+---------------+-----------------+
only showi

/home/pliu/git/ConstanceDataPlatform/cdp_venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pliu/git/ConstanceDataPlatform/cdp_venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
